In [1]:
# import subprocess
# subprocess.call(['sh', './install_packages.sh'])
import taichi as ti
import numpy as np
import math
from sympy import inverse_mellin_transform
from pyevtk.hl import gridToVTK
import pandas as pd
import vtk
import time
import LBM_parameters as lbm
import json
import LBM_solver as LBM

%load_ext nb_black
ti.init(arch=ti.cpu, dynamic_index=False)
# ti.init(arch=ti.cpu,cpu_max_num_threads=32, dynamic_index=False)

"""Default setting"""
# LBM parameters
Q = 19
dim = 3

# Adhesion and cohesion parameters
G_1s = -0.4
G_12 = 4.0
G_2s = -G_1s
G_11 = 0
G_22 = 0

# Viscosity and initial density
niu_1 = 0.2
niu_2 = 0.2
rhos = 3
rho_2 = 2
rho_1 = 2

tau_1f, tau_2f, S_dig_np_1, S_dig_np_2 = lbm.update_S(niu_1, niu_2)

# Injection control
inject_period = 100
density_increment = 0.0

# Pressure and pseudo density
IniPerturbRate = 1
carn_star = False  # if True, the fluid(s) has multiphase
T_Tc = 0.7
rhol_spinodal = 0.2724
rhog_spinodal = 0.0484

# Output frequency
vtk_dstep = 50
stat_dstep = 50
vtk_path = "./vtks"
nb_steps = 1000

# Dimension
lx = ly = lz = 100

# Input files
solid_np = np.zeros((lx, ly, lz), dtype=np.int8)
nb_solid = 0

# Operator type:
MRT = True

# Multicomponent or singlecomponent
MC = False

# Allocation of sparse memory
sparse = True

[Taichi] version 1.1.3, llvm 10.0.0, commit 1262a70a, linux, python 3.10.9


Error: Can not import avx core while this file exists: /home/amber/.local/lib/python3.10/site-packages/paddle/fluid/core_avx.so


[I 01/24/23 16:12:34.863 37573] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=x64


<IPython.core.display.Javascript object>

In [2]:
LBM.read_json("./mcmp.json")
# LBM.read_json("./input_MRT_MCSP")

print(LBM.rho_1, LBM.rho_2, LBM.rhos)

FileNotFoundError: [Errno 2] No such file or directory: './mcmp.json'

<IPython.core.display.Javascript object>

In [ ]:
for x in range(LBM.lx):
    for y in range(LBM.ly):
        LBM.solid_np[x][y][LBM.lz - 1] = 1
        LBM.solid_np[x][y][0] = 1
LBM.nb_solid = np.count_nonzero(LBM.solid_np)

In [ ]:
if LBM.MC:
    lbm_solver = LBM.D3Q19_MC(sparse_mem=sparse)
else:
    print("Run single-compont solver")
    lbm_solver = LBM.D3Q19_SC(sparse_mem=sparse)

lbm_solver.init_field()

if LBM.MC:
    print("{} fluid nodes have been activated!".format(lbm_solver.activity_checking()))
    print(
        "{} fluid nodes in the computational domain!".format(
            LBM.lx * LBM.ly * LBM.lz - LBM.nb_solid
        )
    )

In [ ]:
# begin = time.time()
# nb_liq_nodes = lbm_solver.place_fluid_sphere(50, 50, 15, 16, 7.8)
lbm_solver.run(LBM.nb_steps)
# print("Time elapses for 100 steps using non-sparse memory is ", time.time() - begin)

In [ ]:
G = -1
T_Tc = 0.7


def Press(rho_value):
    #     a = 1.0
    #     b = 4.0
    #     R = 1.0
    #     Tc = 0.0943

    a = 0.09926
    b = 0.18727
    R = 0.2
    Tc = 1.0

    T = T_Tc * Tc
    eta = b * rho_value / 4.0
    eta2 = eta * eta
    eta3 = eta2 * eta
    rho2 = rho_value * rho_value
    one_minus_eta = 1.0 - eta
    one_minus_eta3 = one_minus_eta * one_minus_eta * one_minus_eta
    return rho_value * R * T * (1 + eta + eta2 - eta3) / one_minus_eta3 - a * rho2


def Psi(rho_value):
    cs2 = 1.0 / 3.0
    p = Press(rho_value)
    return np.sqrt(-2.0 * (p - cs2 * rho_value) / cs2)


print(Psi(7) / 4.0149)

In [ ]:
import matplotlib.pyplot as plt

rho_s = 0.4

a_0 = 0.005
rho_10 = - 0.001/np.log10(a_0)
rho_20 = rho_s/4

rho = np.linspace(0, rho_s, 100)

a = 1.0 - np.exp(-rho / rho_20)

# b = a_0 - np.exp(-rho / rho_10)
plt.plot(rho, a)
# plt.plot(rho, b)

print(1.0 - np.exp(-4/ rho_20))


In [ ]:
rho = 0.001

a = 1.0 - np.exp(-rho / rho_20)
b = a_0 - np.exp(-rho / rho_10)
print(a, b, -rho_10 * np.log(a_0))